#### NLP Processing for our unstruvtured data
Now we have our four features but by observing above table we have some string data available and as we know that machines can not understand these strings so we have transform these string respresented data in to numerical form.

Here we can apply one hot encode to the subreddit but applying to title will not give proper result so for the title, we do nlp on that and trying to get one global matrix for title.

To do NLP on title, we have to follow the particular pipeline to get the major words from the title. We are applying pipeline given as below.

- REMOVE PUNCTUATION
- TOKENIZATION
- REMOVING STOPWORDS
- STEMMING (IF NEEDED TO IMPROVE ACCURACY)
- VECTORIZING

In [4]:
import pandas as pd
import os

In [5]:
def preprocess(df):
    
    df = df[df.columns.drop(list(df.filter(regex='^Cat')))]
    df = df[df['Date'] != '27/06/2001']  #removing the date
    df[(df['Subject'] != 'RE:') & (df['Subject'] != 'FW:') & (df['Subject'] != 'Re:')]  #removing the max same subjects
    del df['Unnamed: 0']
    return df

In [6]:
DATA = 'data' #https://data.world/brianray/enron-email-dataset

FILENAMES = [os.path.join(DATA, filename) for filename in os.listdir(DATA)]
df = pd.read_csv(FILENAMES[0])
df = preprocess(df)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head()

,Message-ID,Date,From,To,Subject,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,content,user,labeled
0,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 23:39:00,frozenset({'phillip.allen@enron.com'}),frozenset({'tim.belden@enron.com'}),NaN,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast,allen-p,False
1,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 20:51:00,frozenset({'phillip.allen@enron.com'}),frozenset({'john.lavorato@enron.com'}),Re:,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p,False
2,<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 10:00:00,frozenset({'phillip.allen@enron.com'}),frozenset({'leah.arsdall@enron.com'}),Re: test,Phillip K Allen,Leah Van Arsdall,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p,False
3,<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 13:13:00,frozenset({'phillip.allen@enron.com'}),frozenset({'randall.gay@enron.com'}),NaN,Phillip K Allen,Randall L Gay,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy, Can you send me a schedule of the salar...",allen-p,False
4,<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 12:07:00,frozenset({'phillip.allen@enron.com'}),frozenset({'greg.piper@enron.com'}),Re: Hello,Phillip K Allen,Greg Piper,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p,False


In [8]:
dfEmail = df[['Subject', 'content']]

In [9]:
dfEmail.head()

,Subject,content
0,NaN,Here is our forecast
1,Re:,Traveling to have a business meeting takes the...
2,Re: test,test successful. way to go!!!
3,NaN,"Randy, Can you send me a schedule of the salar..."
4,Re: Hello,Let's shoot for Tuesday at 11:45.


##### Remove Punctuation
In this method, We are removing all the following the punctuations from content and subject

In [10]:

import string

print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [11]:
def remove_punc(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

''' python is case sensetive for that A and a is diffrent thats why lower()'''
dfEmail['content_removed_punctuation'] = dfEmail['content'].apply(lambda x: remove_punc(str(x).lower()))
dfEmail['Subject_removed_punctuation'] = dfEmail['Subject'].apply(lambda x: remove_punc(str(x).lower()))
dfEmail.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Subject,content,content_removed_punctuation,Subject_removed_punctuation
0,NaN,Here is our forecast,here is our forecast,nan
1,Re:,Traveling to have a business meeting takes the...,traveling to have a business meeting takes the...,re
2,Re: test,test successful. way to go!!!,test successful way to go,re test
3,NaN,"Randy, Can you send me a schedule of the salar...",randy can you send me a schedule of the salary...,nan
4,Re: Hello,Let's shoot for Tuesday at 11:45.,lets shoot for tuesday at 1145,re hello


We can see the difference in text by navigationg the columns

##### Tokenization
Tokenization is the act of breaking up a sequence of strings into pieces such as words, keywords, phrases, symbols and other elements called tokens. Tokens can be individual words, phrases or even whole sentences. In the process of tokenization, some characters like punctuation marks are discarded.

In [12]:
import re


def tokenize(text):
    # Split word non word
    tokens = re.split('\W+', text)
    return tokens

##EACH ROW
dfEmail['Subject_tokenize'] = dfEmail['Subject_removed_punctuation'].apply(lambda x: tokenize(x))
dfEmail['content_tokenize'] = dfEmail['content_removed_punctuation'].apply(lambda x: tokenize(x))
del dfEmail['content_removed_punctuation']
del dfEmail['Subject_removed_punctuation']
dfEmail.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,Subject,content,Subject_tokenize,content_tokenize
0,NaN,Here is our forecast,[nan],"[here, is, our, forecast]"
1,Re:,Traveling to have a business meeting takes the...,[re],"[traveling, to, have, a, business, meeting, ta..."
2,Re: test,test successful. way to go!!!,"[re, test]","[test, successful, way, to, go]"
3,NaN,"Randy, Can you send me a schedule of the salar...",[nan],"[randy, can, you, send, me, a, schedule, of, t..."
4,Re: Hello,Let's shoot for Tuesday at 11:45.,"[re, hello]","[lets, shoot, for, tuesday, at, 1145]"


We can see that each words in the sentence a

##### Remove StepWords
StepWords are those which are words which does not play importance in sentences. Like I am going to watch movie. where 'I', 'am' and 'to' plays part of the stopwords.

Here we will use nltk package to remvoe stopwords.

In [13]:
import nltk
print('NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)')

print('UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED')
#nltk.download()
stopword = nltk.corpus.stopwords.words('english')

NLTK (DOWNLOAD ALL PACKAGES TO PERFORM NLP OPERATION)
UNCOMMENT FOLLOWING LINE To GET NLTK DOWNLOADED


In [16]:
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]
    return text

dfEmail['Subject_nostopwords'] = dfEmail['Subject_tokenize'].apply(lambda x: remove_stopwords(x))
dfEmail['content_nostopwords'] = dfEmail['content_tokenize'].apply(lambda x: remove_stopwords(x))
del dfEmail['Subject_tokenize']
del dfEmail['content_tokenize']
dfEmail.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Subject,content,Subject_nostopwords,content_nostopwords
0,NaN,Here is our forecast,[nan],[forecast]
1,Re:,Traveling to have a business meeting takes the...,[],"[traveling, business, meeting, takes, fun, tri..."
2,Re: test,test successful. way to go!!!,[test],"[test, successful, way, go]"
3,NaN,"Randy, Can you send me a schedule of the salar...",[nan],"[randy, send, schedule, salary, level, everyon..."
4,Re: Hello,Let's shoot for Tuesday at 11:45.,[hello],"[lets, shoot, tuesday, 1145]"
